# Projekt SSN (Sztuczne sieci neuronowe)
### Autorzy: Aleksander Folfas, Zosia Jaworksa, Maciej Komosa, Kajetan Łodej, Natalia Łyś

### Opis
Lorem ipsum. <- tutaj ma być opis

### Przegląd literatury
Lorem ipsum. <- tutaj przeglad literatury

## Kod sztucznej sieci neuronowej

In [1]:
# Import bibliotek
import pandas as pd
import numpy as np

In [ ]:
# Wczytanie danych

### Kod funkcji aktywacji